# Evaluation of the experts assessment for the questionnaire:
# https://doi.org/10.5281/zenodo.1156293

### Authors of this evaluation

  * Felicidad García-Sánchez (Author of the questionnaire, concepts, analysis)
  * Juan Cruz-Benito (Python code)


In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 999)

from scipy import stats

# import plotly.plotly as py
# from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Bar, Scatter, Figure, Layout
init_notebook_mode(connected=True)
# import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
pd.options.display.max_rows = 500

In [2]:
csvfilepath = 'results-survey131688_complete.csv'

In [3]:
expertsResponses = pd.read_csv(csvfilepath, low_memory=False)

In [4]:
display(expertsResponses.head())

,ID de respuesta,Fecha de envío,Última página,Lenguaje inicial,Fecha de inicio,Fecha de la última acción,Dirección IP,Sexo,Edad,Área de conocimiento,"¿Cuál es tu campo de especialización (Informática, Bellas Artes, Historia del Arte, Comunicación Audiovisual, Ciencias Sociales, etc.)? (Respuesta abierta)",Pertinente,Clara,Relevante,Observaciones - Propuestas,Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Pertinente],Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Clara],Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Relevante],Pregunta: Edad Posibles respuestas: Respuesta abierta [Pertinente],Pregunta: Edad Posibles respuestas: Respuesta abierta [Clara],Pregunta: Edad Posibles respuestas: Respuesta abierta [Relevante],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Pertinente],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Clara],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Relevante],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Pertinente],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Clara],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Relevante],Observaciones - Propuestas.1,Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Pertinente],Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Clara],Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Relevante],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Pertinente],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Clara],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Relevante],"Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Pertinente]","Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Clara]","Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Relevante]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Pertinente]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Clara]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Relevante]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Pertinente]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Clara]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Relevante]",Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV? Posibles respuestas: Nada Menos de una hora 1-2 horas 2-4 horas 4-6 horas Más de 6 horas [Pertinente],Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV? Posibles respuestas: Nada Menos de una hora 1-2 horas 2-4 horas 4-6 horas Más de 6 horas [Cla

In [5]:
NaNValues = expertsResponses.isnull().sum().tolist()
columns = expertsResponses.columns.values.tolist()

print("Total rows: ", expertsResponses['ID de respuesta'].count())

print("\nTotal NaN values per column\n----------")
i=0
for c in columns:
    print(str(columns[i])+": "+str(NaNValues[i]))
    i+=1

Total rows:  5

Total NaN values per column
----------
ID de respuesta: 0
Fecha de envío: 0
Última página: 0
Lenguaje inicial: 0
Fecha de inicio: 0
Fecha de la última acción: 0
Dirección IP: 0
Sexo: 0
Edad: 0
Área de conocimiento: 0
¿Cuál es tu campo de especialización (Informática, Bellas Artes, Historia del Arte, Comunicación Audiovisual, Ciencias Sociales, etc.)? (Respuesta abierta): 0
Pertinente: 0
Clara: 0
Relevante: 0
Observaciones - Propuestas: 2
Pregunta: Sexo     Posibles respuestas:  Masculino 	Femenino 	Otro   [Pertinente]: 0
Pregunta: Sexo     Posibles respuestas:  Masculino 	Femenino 	Otro   [Clara]: 0
Pregunta: Sexo     Posibles respuestas:  Masculino 	Femenino 	Otro   [Relevante]: 0
Pregunta: Edad     Posibles respuestas: Respuesta abierta    [Pertinente]: 0
Pregunta: Edad     Posibles respuestas: Respuesta abierta    [Clara]: 0
Pregunta: Edad     Posibles respuestas: Respuesta abierta    [Relevante]: 0
Pregunta: Nacionalidad     Posibles respuestas: Respuesta abierta   

In [6]:
eResponses = expertsResponses.copy()

eResponsesNum = expertsResponses.copy()

In [7]:
eResponsesNum = eResponsesNum.replace(str('No'),0)
eResponsesNum = eResponsesNum.replace(str('Sí'),1)
eResponsesNum = eResponsesNum.replace(str('NS/NC'),np.NaN)

In [8]:
columns2drop = ['ID de respuesta',
                'Fecha de envío',
                'Última página',
                'Lenguaje inicial',
                'Fecha de inicio',
                'Fecha de la última acción',
                'Dirección IP',
                'Tiempo total',
                'Temporización del grupo: Datos del experto',
                'Temporización de la pregunta: sexo',
                'Temporización de la pregunta: edad',
                'Temporización de la pregunta: areaConocimiento',
                'Temporización de la pregunta: especializacion',
                'Temporización del grupo: Propuesta de invitación',
                'Temporización de la pregunta: invitacionPertinente',
                'Temporización de la pregunta: invitacionClara',
                'Temporización de la pregunta: invitacionRelevante',
                'Temporización de la pregunta: invitacionObservacio',
                'Temporización del grupo: Propuesta de recogida de información personal',
                'Temporización de la pregunta: infoRecogidaSexo',
                'Temporización de la pregunta: infoRecogidaEdad',
                'Temporización de la pregunta: infoRecogidaNacional',
                'Temporización de la pregunta: infoRecogidaEstudios',
                'Temporización de la pregunta: infoRecogidaObservac',
                'Temporización del grupo: Propuesta de cuestionario sobre alfabetización visual. Constructo 1: Recepción de imágenes',
                'Temporización de la pregunta: recepcion01',
                'Temporización de la pregunta: recepcion02',
                'Temporización de la pregunta: recepcion03',
                'Temporización de la pregunta: recepcion04',
                'Temporización de la pregunta: recepcion05',
                'Temporización de la pregunta: recepcion06',
                'Temporización de la pregunta: recepcion07',
                'Temporización de la pregunta: recepcion08',
                'Temporización de la pregunta: recepcion09',
                'Temporización de la pregunta: recepcion10',
                'Temporización de la pregunta: recepcion11',
                'Temporización de la pregunta: recepcion12',
                'Temporización del grupo: Propuesta de cuestionario sobre alfabetización visual. Prosumo (producción y consumo) de imágenes',
                'Temporización de la pregunta: prosumo01',
                'Temporización de la pregunta: prosumo02',
                'Temporización de la pregunta: prosumo03',
                'Temporización de la pregunta: prosumo04',
                'Temporización de la pregunta: prosumo05',
                'Temporización de la pregunta: prosumo06',
                'Temporización de la pregunta: prosumo07',
                'Temporización de la pregunta: prosumo08',
                'Temporización de la pregunta: prosumo09',
                'Temporización de la pregunta: prosumo10',
                'Temporización de la pregunta: prosumo11',
                'Temporización de la pregunta: prosumo12',
                'Temporización de la pregunta: prosumo13',
                'Temporización de la pregunta: prosumo14',
                'Temporización de la pregunta: prosumo15',
                'Temporización de la pregunta: prosumo16',
                'Temporización de la pregunta: prosumo17',
                'Temporización de la pregunta: prosumo18',
                'Temporización de la pregunta: prosumo19',
                'Temporización de la pregunta: prosumo20',
                'Temporización de la pregunta: prosumo21',
                'Temporización de la pregunta: prosumo22',
                'Temporización de la pregunta: prosumo23',
                'Temporización de la pregunta: prosumo24',
                'Temporización de la pregunta: prosumo25',
                'Temporización de la pregunta: prosumo26',
                'Temporización del grupo: Propuesta de cuestionario sobre alfabetización visual. Percepción visual',
                'Temporización de la pregunta: percepcion01',
                'Temporización de la pregunta: percepcion02',
                'Temporización de la pregunta: percepcion03',
                'Temporización de la pregunta: percepcion04',
                'Temporización de la pregunta: percepcion05',
                'Temporización de la pregunta: percepcion06',
                'Temporización de la pregunta: percepcion07',
                'Temporización de la pregunta: percepcion08',
                'Temporización de la pregunta: percepcion09',
                'Temporización de la pregunta: percepcion10',
                'Temporización de la pregunta: percepcion11',
                'Temporización de la pregunta: percepcion12',
                'Temporización de la pregunta: percepcion13',
                'Temporización de la pregunta: percepcion14',
                'Temporización del grupo: Propuesta de cuestionario sobre alfabetización visual. Interpretación de imágenes',
                'Temporización de la pregunta: interpretacion01',
                'Temporización de la pregunta: interpretacion02',
                'Temporización de la pregunta: interpretacion03',
                'Temporización de la pregunta: interpretacion04',
                'Temporización de la pregunta: interpretacion05',
                'Temporización de la pregunta: interpretacion06',
                'Temporización de la pregunta: interpretacion07',
                'Temporización de la pregunta: interpretacion08',
                'Temporización de la pregunta: interpretacion09',
                'Temporización de la pregunta: interpretacion10',
                'Temporización de la pregunta: interpretacion11',
                'Temporización de la pregunta: interpretacion12',
                'Temporización del grupo: Valoración global del instrumento',
                'Temporización de la pregunta: valoracionGlobal01',
                'Temporización de la pregunta: valoracionGlobal02',
                'Temporización de la pregunta: valoracionGlobal03',
                'Temporización de la pregunta: valoracionGlobal04',
                'Temporización de la pregunta: valoracionGlobal05']

eResponses = eResponses.drop(columns2drop, axis=1)
eResponsesNum = eResponsesNum.drop(columns2drop, axis=1)

In [9]:
display(eResponses.head())

,Sexo,Edad,Área de conocimiento,"¿Cuál es tu campo de especialización (Informática, Bellas Artes, Historia del Arte, Comunicación Audiovisual, Ciencias Sociales, etc.)? (Respuesta abierta)",Pertinente,Clara,Relevante,Observaciones - Propuestas,Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Pertinente],Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Clara],Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Relevante],Pregunta: Edad Posibles respuestas: Respuesta abierta [Pertinente],Pregunta: Edad Posibles respuestas: Respuesta abierta [Clara],Pregunta: Edad Posibles respuestas: Respuesta abierta [Relevante],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Pertinente],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Clara],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Relevante],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Pertinente],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Clara],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Relevante],Observaciones - Propuestas.1,Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Pertinente],Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Clara],Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Relevante],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Pertinente],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Clara],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Relevante],"Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Pertinente]","Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Clara]","Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Relevante]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Pertinente]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Clara]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Relevante]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Pertinente]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Clara]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Relevante]",Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV? Posibles respuestas: Nada Menos de una hora 1-2 horas 2-4 horas 4-6 horas Más de 6 horas [Pertinente],Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV? Posibles respuestas: Nada Menos de una hora 1-2 horas 2-4 horas 4-6 horas Más de 6 horas [Clara],Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV? Posibles respuestas: Nada Menos de una hora 1

In [10]:
display(eResponsesNum.head())

,Sexo,Edad,Área de conocimiento,"¿Cuál es tu campo de especialización (Informática, Bellas Artes, Historia del Arte, Comunicación Audiovisual, Ciencias Sociales, etc.)? (Respuesta abierta)",Pertinente,Clara,Relevante,Observaciones - Propuestas,Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Pertinente],Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Clara],Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Relevante],Pregunta: Edad Posibles respuestas: Respuesta abierta [Pertinente],Pregunta: Edad Posibles respuestas: Respuesta abierta [Clara],Pregunta: Edad Posibles respuestas: Respuesta abierta [Relevante],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Pertinente],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Clara],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Relevante],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Pertinente],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Clara],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Relevante],Observaciones - Propuestas.1,Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Pertinente],Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Clara],Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Relevante],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Pertinente],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Clara],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Relevante],"Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Pertinente]","Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Clara]","Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Relevante]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Pertinente]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Clara]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Relevante]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Pertinente]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Clara]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Relevante]",Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV? Posibles respuestas: Nada Menos de una hora 1-2 horas 2-4 horas 4-6 horas Más de 6 horas [Pertinente],Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV? Posibles respuestas: Nada Menos de una hora 1-2 horas 2-4 horas 4-6 horas Más de 6 horas [Clara],Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV? Posibles respuestas: Nada Menos de una hora 1

In [11]:
display(eResponses.shape)

(5, 210)

In [12]:
display(eResponsesNum.shape)

(5, 210)

In [13]:
columns2analyze = eResponses.columns.values.tolist()

for column in columns2analyze:
    try:
        print(column+'\n')
        display(eResponses[column].describe())

        data = [Bar(x=eResponses[column],
                    y=eResponses[column].value_counts())]

        iplot(data)
        # sns.countplot(x=column, data=expertsResponses_work)
        plt.show()
    except:
        print('ERROR: Column: '+column+' not found')

Sexo



count          5
unique         2
top       Hombre
freq           4
Name: Sexo, dtype: object

Edad



count                      5
unique                     2
top       Entre 41 y 50 años
freq                       3
Name: Edad, dtype: object

Área de conocimiento



...

count      5
unique     1
top       Sí
freq       5
Name: Relevancia de las dimensiones. ¿Cada una de las dimensiones colabora en la explicación del proceso de alfabetización visual?   , dtype: object

Puntos fuertes/débiles. Aspectos en los que se basa la solidez del constructo conformado; deficiencias a subsanar.



count                                      5
unique                                     5
top       En general es un buen cuestionario
freq                                       1
Name: Puntos fuertes/débiles. Aspectos en los que se basa la solidez del constructo conformado; deficiencias a subsanar., dtype: object

Observaciones en general



count                                                     5
unique                                                    5
top       Entiendo que el instrumento contará con un may...
freq                                                      1
Name: Observaciones en general, dtype: object

In [14]:
columns2analyze = eResponsesNum.columns.values.tolist()

for column in columns2analyze:
    try:
        print(column+'\n')
        display(eResponsesNum[column].describe())

        data = [Bar(x=eResponsesNum[column],
                    y=eResponsesNum[column].value_counts())]

        iplot(data)
        # sns.countplot(x=column, data=expertsResponses_work)
        plt.show()
    except:
        print('ERROR: Column: '+column+' not found')

Sexo



count          5
unique         2
top       Hombre
freq           4
Name: Sexo, dtype: object

Edad



count                      5
unique                     2
top       Entre 41 y 50 años
freq                       3
Name: Edad, dtype: object

Área de conocimiento



...

count    5.0
mean     1.0
std      0.0
min      1.0
25%      1.0
50%      1.0
75%      1.0
max      1.0
Name: Relevancia de las dimensiones. ¿Cada una de las dimensiones colabora en la explicación del proceso de alfabetización visual?   , dtype: float64

Puntos fuertes/débiles. Aspectos en los que se basa la solidez del constructo conformado; deficiencias a subsanar.



count                                      5
unique                                     5
top       En general es un buen cuestionario
freq                                       1
Name: Puntos fuertes/débiles. Aspectos en los que se basa la solidez del constructo conformado; deficiencias a subsanar., dtype: object

Observaciones en general



count                                                     5
unique                                                    5
top       Entiendo que el instrumento contará con un may...
freq                                                      1
Name: Observaciones en general, dtype: object

## Checking the evaluation of each theoretical construct


### Evaluation of the proposals for the invitation text

In [15]:
columns_invText = ['Pertinente',
                   'Clara', 
                   'Relevante']

display(eResponsesNum[columns_invText].describe())

display(eResponsesNum[columns_invText].stack().describe())

,Pertinente,Clara,Relevante
count,5.0,5.0,5.0
mean,1.0,1.0,1.0
std,0.0,0.0,0.0
min,1.0,1.0,1.0
25%,1.0,1.0,1.0
50%,1.0,1.0,1.0
75%,1.0,1.0,1.0
max,1.0,1.0,1.0


count    15.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
dtype: float64

## Checking the evaluation of each theoretical construct


### Evaluation of the proposals for the personal information retrieval part

In [16]:
'''
Pregunta: Sexo     Posibles respuestas:  Masculino 	Femenino 	Otro   [Pertinente]: 0
Pregunta: Sexo     Posibles respuestas:  Masculino 	Femenino 	Otro   [Clara]: 0
Pregunta: Sexo     Posibles respuestas:  Masculino 	Femenino 	Otro   [Relevante]: 0
Pregunta: Edad     Posibles respuestas: Respuesta abierta    [Pertinente]: 0
Pregunta: Edad     Posibles respuestas: Respuesta abierta    [Clara]: 0
Pregunta: Edad     Posibles respuestas: Respuesta abierta    [Relevante]: 0
Pregunta: Nacionalidad     Posibles respuestas: Respuesta abierta    [Pertinente]: 0
Pregunta: Nacionalidad     Posibles respuestas: Respuesta abierta    [Clara]: 0
Pregunta: Nacionalidad     Posibles respuestas: Respuesta abierta    [Relevante]: 0
Pregunta: Estudios máximos alcanzados     Posibles respuestas:   	Sin terminar Educación Secundaria Obligatoria 	 	 	Terminada Educación Secundaria Obligatoria 	 	 	FP 	 	 	Universitario sin graduar 	 	 	Universitario graduado 	   [Pertinente]: 0
Pregunta: Estudios máximos alcanzados     Posibles respuestas:   	Sin terminar Educación Secundaria Obligatoria 	 	 	Terminada Educación Secundaria Obligatoria 	 	 	FP 	 	 	Universitario sin graduar 	 	 	Universitario graduado 	   [Clara]: 0
Pregunta: Estudios máximos alcanzados     Posibles respuestas:   	Sin terminar Educación Secundaria Obligatoria 	 	 	Terminada Educación Secundaria Obligatoria 	 	 	FP 	 	 	Universitario sin graduar 	 	 	Universitario graduado 	   [Relevante]: 0
'''

columns_persInfo = [8,9,10,
                    11,12,13,
                    14,15,16,
                    17,18,19]

display(eResponsesNum.iloc[:,columns_persInfo].describe())

display(eResponsesNum.iloc[:,columns_persInfo].stack().describe())

,Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Pertinente],Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Clara],Pregunta: Sexo Posibles respuestas: Masculino Femenino Otro [Relevante],Pregunta: Edad Posibles respuestas: Respuesta abierta [Pertinente],Pregunta: Edad Posibles respuestas: Respuesta abierta [Clara],Pregunta: Edad Posibles respuestas: Respuesta abierta [Relevante],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Pertinente],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Clara],Pregunta: Nacionalidad Posibles respuestas: Respuesta abierta [Relevante],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Pertinente],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Clara],Pregunta: Estudios máximos alcanzados Posibles respuestas: Sin terminar Educación Secundaria Obligatoria Terminada Educación Secundaria Obligatoria FP Universitario sin graduar Universitario graduado [Relevante]
count,5.0,5.000000,5.0,5.0,5.000000,5.0,5.0,5.000000,5.0,5.0,5.0,5.000000
mean,1.0,0.800000,1.0,1.0,0.800000,1.0,1.0,0.800000,1.0,1.0,1.0,0.800000
std,0.0,0.447214,0.0,0.0,0.447214,0.0,0.0,0.447214,0.0,0.0,0.0,0.447214
min,1.0,0.000000,1.0,1.0,0.000000,1.0,1.0,0.000000,1.0,1.0,1.0,0.000000
25%,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000
50%,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000
75%,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000
max,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000


count    60.000000
mean      0.933333
std       0.251549
min       0.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
dtype: float64

## Checking the evaluation of each theoretical construct


### Evaluation of the proposals for the image reception construct

In [17]:
'''
Pregunta: Clasifica tu grado de visión  Posibles respuestas   	Muy malo 	 	 	Malo 	 	 	Regular 	 	 	Bueno 	 	 	Muy bueno [Pertinente]: 0
Pregunta: Clasifica tu grado de visión  Posibles respuestas   	Muy malo 	 	 	Malo 	 	 	Regular 	 	 	Bueno 	 	 	Muy bueno [Clara]: 0
Pregunta: Clasifica tu grado de visión  Posibles respuestas   	Muy malo 	 	 	Malo 	 	 	Regular 	 	 	Bueno 	 	 	Muy bueno [Relevante]: 0
Pregunta: ¿Utilizas gafas o lentes de contacto?  Posibles respuestas   	Sí 	 	 	No [Pertinente]: 0
Pregunta: ¿Utilizas gafas o lentes de contacto?  Posibles respuestas   	Sí 	 	 	No [Clara]: 0
Pregunta: ¿Utilizas gafas o lentes de contacto?  Posibles respuestas   	Sí 	 	 	No [Relevante]: 0
Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)?  Posibles respuestas   	Sí 	 	 	No [Pertinente]: 0
Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)?  Posibles respuestas   	Sí 	 	 	No [Clara]: 0
Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)?  Posibles respuestas   	Sí 	 	 	No [Relevante]: 0
Pregunta: Si has contestado "Sí" a la pregunta anterior, indica cuál.  Posibles respuestas: Respuesta abierta [Pertinente]: 0
Pregunta: Si has contestado "Sí" a la pregunta anterior, indica cuál.  Posibles respuestas: Respuesta abierta [Clara]: 0
Pregunta: Si has contestado "Sí" a la pregunta anterior, indica cuál.  Posibles respuestas: Respuesta abierta [Relevante]: 0
Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente?  Posibles respuestas:   	Nunca 	 	 	Ocasionalmente 	 	 	Semanalmente 	 	 	Diariamente 	 	 	Varias veces al día [Pertinente]: 0
Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente?  Posibles respuestas:   	Nunca 	 	 	Ocasionalmente 	 	 	Semanalmente 	 	 	Diariamente 	 	 	Varias veces al día [Clara]: 0
Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente?  Posibles respuestas:   	Nunca 	 	 	Ocasionalmente 	 	 	Semanalmente 	 	 	Diariamente 	 	 	Varias veces al día [Relevante]: 0
Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV?  Posibles respuestas:   	Nada 	 	 	Menos de una hora 	 	 	1-2 horas 	 	 	2-4 horas 	 	 	4-6 horas 	 	 	Más de 6 horas [Pertinente]: 0
Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV?  Posibles respuestas:   	Nada 	 	 	Menos de una hora 	 	 	1-2 horas 	 	 	2-4 horas 	 	 	4-6 horas 	 	 	Más de 6 horas [Clara]: 0
Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV?  Posibles respuestas:   	Nada 	 	 	Menos de una hora 	 	 	1-2 horas 	 	 	2-4 horas 	 	 	4-6 horas 	 	 	Más de 6 horas [Relevante]: 0
Pregunta: ¿Qué tipo de aplicaciones utilizas en tu ordenador/tablet/móvil?     Posibles respuestas: (Marca con una X. Respuesta múltiple)   	Apps de contenidos audiovisuales (Netflix, HBO, etc.) 	 	 	Aplicaciones de mensajería (Whatsapp, Facebook Messenger, etc.) 	 	 	Bibliotecas de fotos online (Google Photos, iPhoto, etc.) 	 	 	Redes sociales (Facebook, Instagram, Twitter, etc.). 	 	 	Aplicaciones de vídeo (Youtube, Vimeo, etc.). 	 	 	Otras apps similares a las anteriores (introduce el nombre) [Pertinente]: 0
Pregunta: ¿Qué tipo de aplicaciones utilizas en tu ordenador/tablet/móvil?     Posibles respuestas: (Marca con una X. Respuesta múltiple)   	Apps de contenidos audiovisuales (Netflix, HBO, etc.) 	 	 	Aplicaciones de mensajería (Whatsapp, Facebook Messenger, etc.) 	 	 	Bibliotecas de fotos online (Google Photos, iPhoto, etc.) 	 	 	Redes sociales (Facebook, Instagram, Twitter, etc.). 	 	 	Aplicaciones de vídeo (Youtube, Vimeo, etc.). 	 	 	Otras apps similares a las anteriores (introduce el nombre) [Clara]: 0
Pregunta: ¿Qué tipo de aplicaciones utilizas en tu ordenador/tablet/móvil?     Posibles respuestas: (Marca con una X. Respuesta múltiple)   	Apps de contenidos audiovisuales (Netflix, HBO, etc.) 	 	 	Aplicaciones de mensajería (Whatsapp, Facebook Messenger, etc.) 	 	 	Bibliotecas de fotos online (Google Photos, iPhoto, etc.) 	 	 	Redes sociales (Facebook, Instagram, Twitter, etc.). 	 	 	Aplicaciones de vídeo (Youtube, Vimeo, etc.). 	 	 	Otras apps similares a las anteriores (introduce el nombre) [Relevante]: 0
Pregunta: Cuando utilizas Internet… ¿recibes y envías imágenes habitualmente?     Posibles respuestas:   	Nunca 	 	 	Pocas veces 	 	 	Ni mucho ni poco 	 	 	Bastantes veces 	 	 	Recibo y envío imágenes de forma constante. [Pertinente]: 0
Pregunta: Cuando utilizas Internet… ¿recibes y envías imágenes habitualmente?     Posibles respuestas:   	Nunca 	 	 	Pocas veces 	 	 	Ni mucho ni poco 	 	 	Bastantes veces 	 	 	Recibo y envío imágenes de forma constante. [Clara]: 0
Pregunta: Cuando utilizas Internet… ¿recibes y envías imágenes habitualmente?     Posibles respuestas:   	Nunca 	 	 	Pocas veces 	 	 	Ni mucho ni poco 	 	 	Bastantes veces 	 	 	Recibo y envío imágenes de forma constante. [Relevante]: 0
Pregunta: Durante ese uso, ¿cuántas imágenes consumes (ves y compartes)?     Posibles respuestas:   	Ninguna 	 	 	Pocas 	 	 	Ni muchas ni pocas 	 	 	Bastantes 	 	 	La mayoría del contenido que consumo son imágenes [Pertinente]: 0
Pregunta: Durante ese uso, ¿cuántas imágenes consumes (ves y compartes)?     Posibles respuestas:   	Ninguna 	 	 	Pocas 	 	 	Ni muchas ni pocas 	 	 	Bastantes 	 	 	La mayoría del contenido que consumo son imágenes [Clara]: 0
Pregunta: Durante ese uso, ¿cuántas imágenes consumes (ves y compartes)?     Posibles respuestas:   	Ninguna 	 	 	Pocas 	 	 	Ni muchas ni pocas 	 	 	Bastantes 	 	 	La mayoría del contenido que consumo son imágenes [Relevante]: 0
Pregunta: ¿Cuál es tu herramienta habitual de consumo de imágenes?     Posibles respuestas:   	Móvil 	 	 	Televisión 	 	 	Ordenador 	 	 	Tableta 	 	 	Otros (libros y periódicos en papel, marquesinas, museos...) [Pertinente]: 0
Pregunta: ¿Cuál es tu herramienta habitual de consumo de imágenes?     Posibles respuestas:   	Móvil 	 	 	Televisión 	 	 	Ordenador 	 	 	Tableta 	 	 	Otros (libros y periódicos en papel, marquesinas, museos...) [Clara]: 0
Pregunta: ¿Cuál es tu herramienta habitual de consumo de imágenes?     Posibles respuestas:   	Móvil 	 	 	Televisión 	 	 	Ordenador 	 	 	Tableta 	 	 	Otros (libros y periódicos en papel, marquesinas, museos...) [Relevante]: 0
Pregunta: ¿Cada cuánto tiempo consumes imágenes?     Posibles respuestas:   	Nunca 	 	 	Dos o tres veces por semana 	 	 	Todos/casi todos los días 	 	 	A diario [Pertinente]: 0
Pregunta: ¿Cada cuánto tiempo consumes imágenes?     Posibles respuestas:   	Nunca 	 	 	Dos o tres veces por semana 	 	 	Todos/casi todos los días 	 	 	A diario [Clara]: 0
Pregunta: ¿Cada cuánto tiempo consumes imágenes?     Posibles respuestas:   	Nunca 	 	 	Dos o tres veces por semana 	 	 	Todos/casi todos los días 	 	 	A diario [Relevante]: 0
Observaciones - Propuestas.2: 1
'''

columns_ImagRecep = [21,22,23,
                     24,25,26,
                     27,28,29,
                     30,31,31,
                     33,34,35,
                     36,37,38,
                     39,40,41,
                     42,43,44,
                     45,46,47,
                     48,49,50,
                     51,52,53]

display(eResponsesNum.iloc[:,columns_ImagRecep].describe())

display(eResponsesNum.iloc[:,columns_ImagRecep].stack().describe())

,Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Pertinente],Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Clara],Pregunta: Clasifica tu grado de visión Posibles respuestas Muy malo Malo Regular Bueno Muy bueno [Relevante],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Pertinente],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Clara],Pregunta: ¿Utilizas gafas o lentes de contacto? Posibles respuestas Sí No [Relevante],"Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Pertinente]","Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Clara]","Pregunta: ¿Tienes alguna carencia visual que afecte a la recepción de formas, colores, distancias... (daltonismo, etc.)? Posibles respuestas Sí No [Relevante]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Pertinente]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Clara]","Pregunta: Si has contestado ""Sí"" a la pregunta anterior, indica cuál. Posibles respuestas: Respuesta abierta [Clara]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Pertinente]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Clara]","Pregunta: ¿Tienes acceso a dispositivos (ordenador, móvil) conectados a Internet habitualmente? Posibles respuestas: Nunca Ocasionalmente Semanalmente Diariamente Varias veces al día [Relevante]",Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV? Posibles respuestas: Nada Menos de una hora 1-2 horas 2-4 horas 4-6 horas Más de 6 horas [Pertinente],Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV? Posibles respuestas: Nada Menos de una hora 1-2 horas 2-4 horas 4-6 horas Más de 6 horas [Clara],Pregunta: ¿Cuánto tiempo al día usas el móvil/tableta/ordenador/TV? Posibles respuestas: Nada Menos de una hora 1-2 horas 2-4 horas 4-6 horas Más de 6 horas [Relevante],"Pregunta: ¿Qué tipo de aplicaciones utilizas en tu ordenador/tablet/móvil? Posibles respuestas: (Marca con una X. Respuesta múltiple) Apps de contenidos audiovisuales (Netflix, HBO, etc.) Aplicaciones de mensajería (Whatsapp, Facebook Messenger, etc.) Bibliotecas de fotos online (Google Photos, iPhoto, etc.) Redes sociales (Facebook, Instagram, Twitter, etc.). Aplicaciones de vídeo (Youtube, Vimeo, etc.). Otras apps similares a las anteriores (introduce el nombre) [Pertinente]","Pregunta: ¿Qué tipo de aplicaciones utilizas en tu ordenador/tablet/móvil? Posibles respuestas: (Marca con una X. Respuesta múltiple) Apps de contenidos audiovisuales (Netflix, HBO, etc.) Aplicaciones de mensajería (Whatsapp, Facebook Messenger, etc.) Bibliotecas de fotos online (Google Photos, iPhoto, etc.) Redes sociales (Facebook, Instagram, Twitter, etc.). Aplicaciones de vídeo (Youtube, Vimeo, etc.). Otras apps similares a las anteriores (introduce el nombre) [Clara]","Pregunta: ¿Qué tipo de aplicaciones utilizas en tu ordenador/tablet/móvil? Posibles respuestas: (Marca con una X. Respuesta múltiple) Apps de contenidos audiovisuales (Netflix, HBO, etc.) Aplicaciones de mensajería (Whatsapp, Facebook Messenger, etc.) Bibliotecas de fotos online (Google Photos, iPhoto, etc.) Redes sociales (Facebook, Instagram, Twitter, etc.). Aplicaciones de vídeo (Youtube, Vimeo, etc.). Otras apps similares a las anteriores (introduce el nombre) [Relevante]",Pregunta: Cuando utilizas Internet

count    161.000000
mean       0.981366
std        0.135649
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
dtype: float64

## Checking the evaluation of each theoretical construct


### Evaluation of the proposals for the prosumption construct

In [18]:
'''
Pregunta: Cuando haces una foto, ¿qué tipo de dispositivo de captación usas?  Posibles respuestas:  Cámara de fotos 	Teléfono móvil 	Tablet 	Otros dispositivos ¿cuál/es? ____________   [Pertinente]: 0
Pregunta: Cuando haces una foto, ¿qué tipo de dispositivo de captación usas?  Posibles respuestas:  Cámara de fotos 	Teléfono móvil 	Tablet 	Otros dispositivos ¿cuál/es? ____________   [Clara]: 0
Pregunta: Cuando haces una foto, ¿qué tipo de dispositivo de captación usas?  Posibles respuestas:  Cámara de fotos 	Teléfono móvil 	Tablet 	Otros dispositivos ¿cuál/es? ____________   [Relevante]: 0
Pregunta: ¿Con qué frecuencia haces fotografías?  Posibles respuestas:   	Nunca 	 	 	Una vez al mes 	 	 	Dos o tres veces por semana 	 	 	A diario 	   [Pertinente]: 0
Pregunta: ¿Con qué frecuencia haces fotografías?  Posibles respuestas:   	Nunca 	 	 	Una vez al mes 	 	 	Dos o tres veces por semana 	 	 	A diario 	   [Clara]: 0
Pregunta: ¿Con qué frecuencia haces fotografías?  Posibles respuestas:   	Nunca 	 	 	Una vez al mes 	 	 	Dos o tres veces por semana 	 	 	A diario 	   [Relevante]: 0
Pregunta: En caso afirmativo, ¿compartes habitualmente las fotografías que tomas?  Posibles respuestas:   	Nunca 	 	 	Alguna vez 	 	 	La mayor parte de las veces 	 	 	Siempre 	   [Pertinente]: 0
Pregunta: En caso afirmativo, ¿compartes habitualmente las fotografías que tomas?  Posibles respuestas:   	Nunca 	 	 	Alguna vez 	 	 	La mayor parte de las veces 	 	 	Siempre 	   [Clara]: 0
Pregunta: En caso afirmativo, ¿compartes habitualmente las fotografías que tomas?  Posibles respuestas:   	Nunca 	 	 	Alguna vez 	 	 	La mayor parte de las veces 	 	 	Siempre 	   [Relevante]: 0
Pregunta: En tu entorno, ¿las personas que utilizan la tecnología crean imágenes y las comparten?  Posibles respuestas:   	Nunca 	 	 	En raras ocasiones 	 	 	Ni mucho ni poco 	 	 	Bastantes veces 	 	 	Casi siempre o siempre 	   [Pertinente]: 0
Pregunta: En tu entorno, ¿las personas que utilizan la tecnología crean imágenes y las comparten?  Posibles respuestas:   	Nunca 	 	 	En raras ocasiones 	 	 	Ni mucho ni poco 	 	 	Bastantes veces 	 	 	Casi siempre o siempre 	   [Clara]: 0
Pregunta: En tu entorno, ¿las personas que utilizan la tecnología crean imágenes y las comparten?  Posibles respuestas:   	Nunca 	 	 	En raras ocasiones 	 	 	Ni mucho ni poco 	 	 	Bastantes veces 	 	 	Casi siempre o siempre 	   [Relevante]: 0
Pregunta: Cuando compartes imágenes: ¿habitualmente las recibes de otras personas?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre 	   [Pertinente]: 0
Pregunta: Cuando compartes imágenes: ¿habitualmente las recibes de otras personas?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre 	   [Clara]: 0
Pregunta: Cuando compartes imágenes: ¿habitualmente las recibes de otras personas?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre 	   [Relevante]: 0
Pregunta: Cuando compartes imágenes, ¿habitualmente las envías a otras personas?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre 	   [Pertinente]: 0
Pregunta: Cuando compartes imágenes, ¿habitualmente las envías a otras personas?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre 	   [Clara]: 0
Pregunta: Cuando compartes imágenes, ¿habitualmente las envías a otras personas?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre 	   [Relevante]: 0
Pregunta: Cuando envías una imagen, ¿se trata de una imagen que previamente has recibido o esta ha sido generada por ti?  Posibles respuestas:   	Siempre reenvío imágenes que me envían 	 	 	Muchas veces reenvío imágenes que me envían 	 	 	A veces envío imágenes que hago yo y otras veces no, la proporción es equilibrada 	 	 	Muchas veces envío imágenes que hago yo 	 	 	Siempre mando imágenes que hago yo 	   [Pertinente]: 0
Pregunta: Cuando envías una imagen, ¿se trata de una imagen que previamente has recibido o esta ha sido generada por ti?  Posibles respuestas:   	Siempre reenvío imágenes que me envían 	 	 	Muchas veces reenvío imágenes que me envían 	 	 	A veces envío imágenes que hago yo y otras veces no, la proporción es equilibrada 	 	 	Muchas veces envío imágenes que hago yo 	 	 	Siempre mando imágenes que hago yo 	   [Clara]: 0
Pregunta: Cuando envías una imagen, ¿se trata de una imagen que previamente has recibido o esta ha sido generada por ti?  Posibles respuestas:   	Siempre reenvío imágenes que me envían 	 	 	Muchas veces reenvío imágenes que me envían 	 	 	A veces envío imágenes que hago yo y otras veces no, la proporción es equilibrada 	 	 	Muchas veces envío imágenes que hago yo 	 	 	Siempre mando imágenes que hago yo 	   [Relevante]: 0
Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado?  Posibles respuestas:  Sí 	No   [Pertinente]: 0
Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado?  Posibles respuestas:  Sí 	No   [Clara]: 0
Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado?  Posibles respuestas:  Sí 	No   [Relevante]: 0
Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones?  Posibles respuestas:   	No soy fan de nadie a quien admire 	 	 	Soy fan, pero no interactúas con sus publicaciones 	 	 	A veces interactúo con sus publicaciones en forma de like 	 	 	Soy fan y me gusta relacionarme con ellos de vez en cuando 	 	 	Soy fan y siempre hago comentarios en las publicaciones que me interesan  	  	   [Pertinente]: 0
Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones?  Posibles respuestas:   	No soy fan de nadie a quien admire 	 	 	Soy fan, pero no interactúas con sus publicaciones 	 	 	A veces interactúo con sus publicaciones en forma de like 	 	 	Soy fan y me gusta relacionarme con ellos de vez en cuando 	 	 	Soy fan y siempre hago comentarios en las publicaciones que me interesan  	  	   [Clara]: 0
Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones?  Posibles respuestas:   	No soy fan de nadie a quien admire 	 	 	Soy fan, pero no interactúas con sus publicaciones 	 	 	A veces interactúo con sus publicaciones en forma de like 	 	 	Soy fan y me gusta relacionarme con ellos de vez en cuando 	 	 	Soy fan y siempre hago comentarios en las publicaciones que me interesan  	  	   [Relevante]: 0
Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre / siempre  	  [Pertinente]: 0
Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre / siempre  	  [Clara]: 0
Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre / siempre  	  [Relevante]: 0
Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Pertinente]: 0
Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Clara]: 0
Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Relevante]: 0
Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre  	  [Pertinente]: 0
Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre  	  [Clara]: 0
Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre  	  [Relevante]: 0
Pregunta: Dentro de tus redes sociales o aplicaciones móviles, ¿se comparte información visual que tu no buscas (campañas de concienciación social, chistes, publicidad...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Pertinente]: 0
Pregunta: Dentro de tus redes sociales o aplicaciones móviles, ¿se comparte información visual que tu no buscas (campañas de concienciación social, chistes, publicidad...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Clara]: 0
Pregunta: Dentro de tus redes sociales o aplicaciones móviles, ¿se comparte información visual que tu no buscas (campañas de concienciación social, chistes, publicidad...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Relevante]: 0
Pregunta: Si encuentras esta información visual sugerida, ¿cómo actúas?  Posibles respuestas:   	No accedo a la información propuesta 	 	 	Accedo a la información propuesta [Pertinente]: 0
Pregunta: Si encuentras esta información visual sugerida, ¿cómo actúas?  Posibles respuestas:   	No accedo a la información propuesta 	 	 	Accedo a la información propuesta [Clara]: 0
Pregunta: Si encuentras esta información visual sugerida, ¿cómo actúas?  Posibles respuestas:   	No accedo a la información propuesta 	 	 	Accedo a la información propuesta [Relevante]: 0
Pregunta: Si tienes que crear una presentación de contenidos...  Posibles respuestas:   	Busco imágenes para ilustrar lo que estoy diciendo. 	 	 	Las genero yo [Pertinente]: 0
Pregunta: Si tienes que crear una presentación de contenidos...  Posibles respuestas:   	Busco imágenes para ilustrar lo que estoy diciendo. 	 	 	Las genero yo [Clara]: 0
Pregunta: Si tienes que crear una presentación de contenidos...  Posibles respuestas:   	Busco imágenes para ilustrar lo que estoy diciendo. 	 	 	Las genero yo [Relevante]: 0
Pregunta: Si las buscas en Internet...  Posibles respuestas:   	Busco en Google Imágenes (o similar). No tengo en cuenta la licencia de la imagen que utilizo 	 	 	Busco en Google Imágenes (o similar). Tengo en cuenta la licencia de la imagen que utilizo 	 	 	Busco en web especializadas en imágenes. Conozco y uso contenidos con licencias [Pertinente]: 0
Pregunta: Si las buscas en Internet...  Posibles respuestas:   	Busco en Google Imágenes (o similar). No tengo en cuenta la licencia de la imagen que utilizo 	 	 	Busco en Google Imágenes (o similar). Tengo en cuenta la licencia de la imagen que utilizo 	 	 	Busco en web especializadas en imágenes. Conozco y uso contenidos con licencias [Clara]: 0
Pregunta: Si las buscas en Internet...  Posibles respuestas:   	Busco en Google Imágenes (o similar). No tengo en cuenta la licencia de la imagen que utilizo 	 	 	Busco en Google Imágenes (o similar). Tengo en cuenta la licencia de la imagen que utilizo 	 	 	Busco en web especializadas en imágenes. Conozco y uso contenidos con licencias [Relevante]: 0
Pregunta: Si las generas tú...  Posibles respuestas:   	Utilizo dispositivos para hacer una foto a lo que necesito (cámara de fotos, móvil, tablet...) 	 	 	Las compongo en programas de edición de imágenes desde otras fotografías 	 	 	Utilizo ambas opciones combinadas 	 	 	Otros: _____________ [Pertinente]: 0
Pregunta: Si las generas tú...  Posibles respuestas:   	Utilizo dispositivos para hacer una foto a lo que necesito (cámara de fotos, móvil, tablet...) 	 	 	Las compongo en programas de edición de imágenes desde otras fotografías 	 	 	Utilizo ambas opciones combinadas 	 	 	Otros: _____________ [Clara]: 0
Pregunta: Si las generas tú...  Posibles respuestas:   	Utilizo dispositivos para hacer una foto a lo que necesito (cámara de fotos, móvil, tablet...) 	 	 	Las compongo en programas de edición de imágenes desde otras fotografías 	 	 	Utilizo ambas opciones combinadas 	 	 	Otros: _____________ [Relevante]: 0
Pregunta: ¿Cuándo prosumes (produces y consumes) fotos?  Posibles respuestas:   	Nunca 	 	 	Una vez al mes 	 	 	Dos o tres veces por semana 	 	 	A diario  	  [Pertinente]: 0
Pregunta: ¿Cuándo prosumes (produces y consumes) fotos?  Posibles respuestas:   	Nunca 	 	 	Una vez al mes 	 	 	Dos o tres veces por semana 	 	 	A diario  	  [Clara]: 0
Pregunta: ¿Cuándo prosumes (produces y consumes) fotos?  Posibles respuestas:   	Nunca 	 	 	Una vez al mes 	 	 	Dos o tres veces por semana 	 	 	A diario  	  [Relevante]: 0
Pregunta: Si produces o consumes imágenes, ¿para qué las usas?  Posibles respuestas (respuesta múltiple):   	Para que se entienda mejor el significado de lo que quiero expresar 	 	 	Para explicar mejor dónde me encuentro 	 	 	Para expresar emociones 	 	 	Para ahorrar palabras 	 	 	Para compartir momentos, 	 	 	Otro___________ [Pertinente]: 0
Pregunta: Si produces o consumes imágenes, ¿para qué las usas?  Posibles respuestas (respuesta múltiple):   	Para que se entienda mejor el significado de lo que quiero expresar 	 	 	Para explicar mejor dónde me encuentro 	 	 	Para expresar emociones 	 	 	Para ahorrar palabras 	 	 	Para compartir momentos, 	 	 	Otro___________ [Clara]: 0
Pregunta: Si produces o consumes imágenes, ¿para qué las usas?  Posibles respuestas (respuesta múltiple):   	Para que se entienda mejor el significado de lo que quiero expresar 	 	 	Para explicar mejor dónde me encuentro 	 	 	Para expresar emociones 	 	 	Para ahorrar palabras 	 	 	Para compartir momentos, 	 	 	Otro___________ [Relevante]: 0
Pregunta: Si has respondido que NUNCA produces o consumes imágenes, ¿por qué no las usas?  Posibles respuestas (respuesta múltiple):   	No me gustan 	 	 	No sé cómo se hacen 	 	 	No uso la tecnología 	 	 	Cuando las comparto no sé qué significan 	 	 	Porque ocupan mucho espacio 	 	 	Me parece inútil 	 	 	Otro______________  	  [Pertinente]: 0
Pregunta: Si has respondido que NUNCA produces o consumes imágenes, ¿por qué no las usas?  Posibles respuestas (respuesta múltiple):   	No me gustan 	 	 	No sé cómo se hacen 	 	 	No uso la tecnología 	 	 	Cuando las comparto no sé qué significan 	 	 	Porque ocupan mucho espacio 	 	 	Me parece inútil 	 	 	Otro______________  	  [Clara]: 0
Pregunta: Si has respondido que NUNCA produces o consumes imágenes, ¿por qué no las usas?  Posibles respuestas (respuesta múltiple):   	No me gustan 	 	 	No sé cómo se hacen 	 	 	No uso la tecnología 	 	 	Cuando las comparto no sé qué significan 	 	 	Porque ocupan mucho espacio 	 	 	Me parece inútil 	 	 	Otro______________  	  [Relevante]: 0
Pregunta: ¿Qué imágenes decides compartir en redes sociales u otros medios tecnológicos?  Posibles respuestas:   	Imágenes que realizo para mantener las redes activas. 	 	 	Imágenes del día a día, que forman parte de mí, que utilizo para expresar sentimientos pero que no pienso únicamente para este fin 	 	 	Imágenes referenciales con el fin de mostrar algo de forma informativa. 	 	 	Imágenes que con las que pretendo causar una reacción (ej. likes) 	 	 	Otras _____________ [Pertinente]: 0
Pregunta: ¿Qué imágenes decides compartir en redes sociales u otros medios tecnológicos?  Posibles respuestas:   	Imágenes que realizo para mantener las redes activas. 	 	 	Imágenes del día a día, que forman parte de mí, que utilizo para expresar sentimientos pero que no pienso únicamente para este fin 	 	 	Imágenes referenciales con el fin de mostrar algo de forma informativa. 	 	 	Imágenes que con las que pretendo causar una reacción (ej. likes) 	 	 	Otras _____________ [Clara]: 0
Pregunta: ¿Qué imágenes decides compartir en redes sociales u otros medios tecnológicos?  Posibles respuestas:   	Imágenes que realizo para mantener las redes activas. 	 	 	Imágenes del día a día, que forman parte de mí, que utilizo para expresar sentimientos pero que no pienso únicamente para este fin 	 	 	Imágenes referenciales con el fin de mostrar algo de forma informativa. 	 	 	Imágenes que con las que pretendo causar una reacción (ej. likes) 	 	 	Otras _____________ [Relevante]: 0
Pregunta: Cuando usas redes sociales, ¿consideras que ser aceptado como miembro de un grupo es más importante que tener autonomía e independencia?  Posibles respuestas:   	Nada importante 	 	 	Poco importante 	 	 	Neutral 	 	 	Importante 	 	 	Muy importante [Pertinente]: 0
Pregunta: Cuando usas redes sociales, ¿consideras que ser aceptado como miembro de un grupo es más importante que tener autonomía e independencia?  Posibles respuestas:   	Nada importante 	 	 	Poco importante 	 	 	Neutral 	 	 	Importante 	 	 	Muy importante [Clara]: 0
Pregunta: Cuando usas redes sociales, ¿consideras que ser aceptado como miembro de un grupo es más importante que tener autonomía e independencia?  Posibles respuestas:   	Nada importante 	 	 	Poco importante 	 	 	Neutral 	 	 	Importante 	 	 	Muy importante [Relevante]: 0
Pregunta: ¿Con quién compartes imágenes generalmente?  Posibles respuestas (respuesta múltiple):   	Amigos 	 	 	Familia 	 	 	Compañeros 	 	 	Con todos mis conocidos 	 	 	Con nadie 	 	 	Otro___________  	  [Pertinente]: 0
Pregunta: ¿Con quién compartes imágenes generalmente?  Posibles respuestas (respuesta múltiple):   	Amigos 	 	 	Familia 	 	 	Compañeros 	 	 	Con todos mis conocidos 	 	 	Con nadie 	 	 	Otro___________  	  [Clara]: 0
Pregunta: ¿Con quién compartes imágenes generalmente?  Posibles respuestas (respuesta múltiple):   	Amigos 	 	 	Familia 	 	 	Compañeros 	 	 	Con todos mis conocidos 	 	 	Con nadie 	 	 	Otro___________  	  [Relevante]: 0
Pregunta: Cuando produces imágenes, ¿piensas en la imagen que quieres crear antes de hacerla?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Pertinente]: 0
Pregunta: Cuando produces imágenes, ¿piensas en la imagen que quieres crear antes de hacerla?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Clara]: 0
Pregunta: Cuando produces imágenes, ¿piensas en la imagen que quieres crear antes de hacerla?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Relevante]: 0
Pregunta: ¿Cuánto de tu consumo de estas imágenes viene de los medios digitales (prensa digital, internet, televisión...) y cuánto por medios analógicos (prensa escrita, revistas, libros...)?  Posibles respuestas:   	Todo de medios digitales 	 	 	Casi todo por medios digitales 	 	 	Por igual de medios digitales y analógicos 	 	 	Casi todo de medios analógicos 	 	 	Todo de medios analógicos [Pertinente]: 0
Pregunta: ¿Cuánto de tu consumo de estas imágenes viene de los medios digitales (prensa digital, internet, televisión...) y cuánto por medios analógicos (prensa escrita, revistas, libros...)?  Posibles respuestas:   	Todo de medios digitales 	 	 	Casi todo por medios digitales 	 	 	Por igual de medios digitales y analógicos 	 	 	Casi todo de medios analógicos 	 	 	Todo de medios analógicos [Clara]: 0
Pregunta: ¿Cuánto de tu consumo de estas imágenes viene de los medios digitales (prensa digital, internet, televisión...) y cuánto por medios analógicos (prensa escrita, revistas, libros...)?  Posibles respuestas:   	Todo de medios digitales 	 	 	Casi todo por medios digitales 	 	 	Por igual de medios digitales y analógicos 	 	 	Casi todo de medios analógicos 	 	 	Todo de medios analógicos [Relevante]: 0
'''

columns_Prosumption = [55,56,57,
                       58,59,60,
                       61,62,63,
                       64,65,66,
                       67,68,69,
                       70,71,72,
                       73,74,75,
                       76,77,78,
                       79,80,81,
                       82,83,84,
                       85,86,87,
                       88,89,90,
                       91,92,93,
                       94,95,96,
                       97,98,99,
                       100,101,102,
                       103,104,105,
                       106,107,108,
                       109,110,111,
                       112,113,114,
                       115,116,117,
                       118,119,120,
                       121,122,123,
                       124,125,126,
                       127,128,129
                    ]

display(eResponsesNum.iloc[:,columns_Prosumption].describe())

display(eResponsesNum.iloc[:,columns_Prosumption].stack().describe())

,"Pregunta: Cuando haces una foto, ¿qué tipo de dispositivo de captación usas? Posibles respuestas: Cámara de fotos Teléfono móvil Tablet Otros dispositivos ¿cuál/es? ____________ [Pertinente]","Pregunta: Cuando haces una foto, ¿qué tipo de dispositivo de captación usas? Posibles respuestas: Cámara de fotos Teléfono móvil Tablet Otros dispositivos ¿cuál/es? ____________ [Clara]","Pregunta: Cuando haces una foto, ¿qué tipo de dispositivo de captación usas? Posibles respuestas: Cámara de fotos Teléfono móvil Tablet Otros dispositivos ¿cuál/es? ____________ [Relevante]",Pregunta: ¿Con qué frecuencia haces fotografías? Posibles respuestas: Nunca Una vez al mes Dos o tres veces por semana A diario [Pertinente],Pregunta: ¿Con qué frecuencia haces fotografías? Posibles respuestas: Nunca Una vez al mes Dos o tres veces por semana A diario [Clara],Pregunta: ¿Con qué frecuencia haces fotografías? Posibles respuestas: Nunca Una vez al mes Dos o tres veces por semana A diario [Relevante],"Pregunta: En caso afirmativo, ¿compartes habitualmente las fotografías que tomas? Posibles respuestas: Nunca Alguna vez La mayor parte de las veces Siempre [Pertinente]","Pregunta: En caso afirmativo, ¿compartes habitualmente las fotografías que tomas? Posibles respuestas: Nunca Alguna vez La mayor parte de las veces Siempre [Clara]","Pregunta: En caso afirmativo, ¿compartes habitualmente las fotografías que tomas? Posibles respuestas: Nunca Alguna vez La mayor parte de las veces Siempre [Relevante]","Pregunta: En tu entorno, ¿las personas que utilizan la tecnología crean imágenes y las comparten? Posibles respuestas: Nunca En raras ocasiones Ni mucho ni poco Bastantes veces Casi siempre o siempre [Pertinente]","Pregunta: En tu entorno, ¿las personas que utilizan la tecnología crean imágenes y las comparten? Posibles respuestas: Nunca En raras ocasiones Ni mucho ni poco Bastantes veces Casi siempre o siempre [Clara]","Pregunta: En tu entorno, ¿las personas que utilizan la tecnología crean imágenes y las comparten? Posibles respuestas: Nunca En raras ocasiones Ni mucho ni poco Bastantes veces Casi siempre o siempre [Relevante]",Pregunta: Cuando compartes imágenes: ¿habitualmente las recibes de otras personas? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente],Pregunta: Cuando compartes imágenes: ¿habitualmente las recibes de otras personas? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara],Pregunta: Cuando compartes imágenes: ¿habitualmente las recibes de otras personas? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Relevante],"Pregunta: Cuando compartes imágenes, ¿habitualmente las envías a otras personas? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente]","Pregunta: Cuando compartes imágenes, ¿habitualmente las envías a otras personas? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara]","Pregunta: Cuando compartes imágenes, ¿habitualmente las envías a otras personas? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Relevante]","Pregunta: Cuando envías una imagen, ¿se trata de una imagen que previamente has recibido o esta ha sido generada por ti? Posibles respuestas: Siempre reenvío imágenes que me envían Muchas veces reenvío imágenes que me envían A veces envío imágenes que hago yo y otras veces no, la proporción es equilibrada Muchas veces envío imágenes que hago yo Siempre mando imágenes que hago yo [Pertinente]","Pregunta: Cuando envías una imagen, ¿se trata de una imagen que previamente has recibido o esta ha sido generada por ti? Posibles respuestas: Siempre reenvío imágenes que me envían Muchas veces reenvío imágenes que me envían A veces envío imágenes que hago yo y otras veces no, la proporción es equilibrada Muchas veces envío imáge

count    359.000000
mean       0.974930
std        0.156555
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
dtype: float64

## Checking the evaluation of each cultural sub-construct


### Evaluation of the cultural items within the prosumption construct

In [19]:
'''
Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado?  Posibles respuestas:  Sí 	No   [Pertinente]: 0
Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado?  Posibles respuestas:  Sí 	No   [Clara]: 0
Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado?  Posibles respuestas:  Sí 	No   [Relevante]: 0

Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones?  Posibles respuestas:   	No soy fan de nadie a quien admire 	 	 	Soy fan, pero no interactúas con sus publicaciones 	 	 	A veces interactúo con sus publicaciones en forma de like 	 	 	Soy fan y me gusta relacionarme con ellos de vez en cuando 	 	 	Soy fan y siempre hago comentarios en las publicaciones que me interesan  	  	   [Pertinente]: 0
Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones?  Posibles respuestas:   	No soy fan de nadie a quien admire 	 	 	Soy fan, pero no interactúas con sus publicaciones 	 	 	A veces interactúo con sus publicaciones en forma de like 	 	 	Soy fan y me gusta relacionarme con ellos de vez en cuando 	 	 	Soy fan y siempre hago comentarios en las publicaciones que me interesan  	  	   [Clara]: 0
Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones?  Posibles respuestas:   	No soy fan de nadie a quien admire 	 	 	Soy fan, pero no interactúas con sus publicaciones 	 	 	A veces interactúo con sus publicaciones en forma de like 	 	 	Soy fan y me gusta relacionarme con ellos de vez en cuando 	 	 	Soy fan y siempre hago comentarios en las publicaciones que me interesan  	  	   [Relevante]: 0

Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre / siempre  	  [Pertinente]: 0
Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre / siempre  	  [Clara]: 0
Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre / siempre  	  [Relevante]: 0

Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Pertinente]: 0
Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Clara]: 0
Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Relevante]: 0

Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre  	  [Pertinente]: 0
Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre  	  [Clara]: 0
Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre  	  [Relevante]: 0
'''

# Checking all cultural items within the construct

print("Evaluation of all cultural items within the construct")

columns_CulturalProsumption = [76,77,78,
                               79,80,81,
                               82,83,84,
                               85,86,87,
                               88,89,90
                              ]

display(eResponsesNum.iloc[:,columns_CulturalProsumption].describe())

display(eResponsesNum.iloc[:,columns_CulturalProsumption].stack().describe())


# Individualism vs. Collectivism // Power distance index

print("Individualism vs. Collectivism // Power distance index")

columns_CulturalProsumption_IndCollect = [76,77,78]

display(eResponsesNum.iloc[:,columns_CulturalProsumption_IndCollect].describe())

display(eResponsesNum.iloc[:,columns_CulturalProsumption_IndCollect].stack().describe())


# Avoidance of uncertainty index

print("'Avoidance of uncertainty index' items")

columns_CulturalProsumption_AvoUncert = [79,80,81,
                                         82,83,84,
                                         85,86,87
                                        ]

display(eResponsesNum.iloc[:,columns_CulturalProsumption_AvoUncert].describe())

display(eResponsesNum.iloc[:,columns_CulturalProsumption_AvoUncert].stack().describe())


# Masculinity vs feminity

print("'Masculinity vs feminity' items")

columns_CulturalProsumption_MasFem = [88,89,90]

display(eResponsesNum.iloc[:,columns_CulturalProsumption_MasFem].describe())

display(eResponsesNum.iloc[:,columns_CulturalProsumption_MasFem].stack().describe())

Evaluation of all cultural items within the construct


,Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado? Posibles respuestas: Sí No [Pertinente],Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado? Posibles respuestas: Sí No [Clara],Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado? Posibles respuestas: Sí No [Relevante],"Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones? Posibles respuestas: No soy fan de nadie a quien admire Soy fan, pero no interactúas con sus publicaciones A veces interactúo con sus publicaciones en forma de like Soy fan y me gusta relacionarme con ellos de vez en cuando Soy fan y siempre hago comentarios en las publicaciones que me interesan [Pertinente]","Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones? Posibles respuestas: No soy fan de nadie a quien admire Soy fan, pero no interactúas con sus publicaciones A veces interactúo con sus publicaciones en forma de like Soy fan y me gusta relacionarme con ellos de vez en cuando Soy fan y siempre hago comentarios en las publicaciones que me interesan [Clara]","Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones? Posibles respuestas: No soy fan de nadie a quien admire Soy fan, pero no interactúas con sus publicaciones A veces interactúo con sus publicaciones en forma de like Soy fan y me gusta relacionarme con ellos de vez en cuando Soy fan y siempre hago comentarios en las publicaciones que me interesan [Relevante]","Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre / siempre [Pertinente]","Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre / siempre [Clara]","Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre / siempre [Relevante]","Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente]","Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara]","Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Relevante]","Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente]","Pregunta: Cuando haces fotos desde cualquiera de los medios a

count    72.000000
mean      0.986111
std       0.117851
min       0.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
dtype: float64

Individualism vs. Collectivism // Power distance index


,Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado? Posibles respuestas: Sí No [Pertinente],Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado? Posibles respuestas: Sí No [Clara],Pregunta: ¿Al reutilizar una imagen puedes redefinir el mensaje que el emisor ha pensado? Posibles respuestas: Sí No [Relevante]
count,5.0,5.000000,5.0
mean,1.0,0.800000,1.0
std,0.0,0.447214,0.0
min,1.0,0.000000,1.0
25%,1.0,1.000000,1.0
50%,1.0,1.000000,1.0
75%,1.0,1.000000,1.0
max,1.0,1.000000,1.0


...

,"Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones? Posibles respuestas: No soy fan de nadie a quien admire Soy fan, pero no interactúas con sus publicaciones A veces interactúo con sus publicaciones en forma de like Soy fan y me gusta relacionarme con ellos de vez en cuando Soy fan y siempre hago comentarios en las publicaciones que me interesan [Pertinente]","Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones? Posibles respuestas: No soy fan de nadie a quien admire Soy fan, pero no interactúas con sus publicaciones A veces interactúo con sus publicaciones en forma de like Soy fan y me gusta relacionarme con ellos de vez en cuando Soy fan y siempre hago comentarios en las publicaciones que me interesan [Clara]","Pregunta: ¿Te gusta seguir a personas que admiras en redes sociales? ¿Eres fan o follower de alguna? ¿En ocasiones escribes comentarios en sus publicaciones? Posibles respuestas: No soy fan de nadie a quien admire Soy fan, pero no interactúas con sus publicaciones A veces interactúo con sus publicaciones en forma de like Soy fan y me gusta relacionarme con ellos de vez en cuando Soy fan y siempre hago comentarios en las publicaciones que me interesan [Relevante]","Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre / siempre [Pertinente]","Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre / siempre [Clara]","Pregunta: Cuando compartes imágenes en redes sociales, ¿las acompañas de texto siguiendo las indicaciones que te sugieren las aplicaciones (etiquetado, explicación de la imagen, texto inspirador...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre / siempre [Relevante]","Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente]","Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara]","Pregunta: En alguna ocasión, aún estando preparado para ello, ¿has dejado de incorporar imágenes en tus mensajes en redes sociales, aplicaciones móviles, etc., porque no entiendes cómo hacerlo? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Relevante]"
count,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


count    44.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
dtype: float64

'Masculinity vs feminity' items


,"Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente]","Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara]","Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Relevante]"
count,5.0,4.0,4.0
mean,1.0,1.0,1.0
std,0.0,0.0,0.0
min,1.0,1.0,1.0
25%,1.0,1.0,1.0
50%,1.0,1.0,1.0
75%,1.0,1.0,1.0
max,1.0,1.0,1.0


count    13.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
dtype: float64

## Checking the evaluation of each theoretical construct


### Evaluation of the proposals for the perception construct

In [20]:
'''
Pregunta: ¿Qué ves en estas imágenes?  Posibles valores de respuesta: Respuesta abierta           [Pertinente]: 0
Pregunta: ¿Qué ves en estas imágenes?  Posibles valores de respuesta: Respuesta abierta           [Clara]: 0
Pregunta: ¿Qué ves en estas imágenes?  Posibles valores de respuesta: Respuesta abierta           [Relevante]: 0
Pregunta: ¿Qué ves en estas imágenes?  Posibles valores de respuesta: Respuesta abierta              [Pertinente]: 0
Pregunta: ¿Qué ves en estas imágenes?  Posibles valores de respuesta: Respuesta abierta              [Clara]: 0
Pregunta: ¿Qué ves en estas imágenes?  Posibles valores de respuesta: Respuesta abierta              [Relevante]: 0
Pregunta: ¿Cuál de las imágenes emparejadas te resulta más atractiva?  Posibles valores de respuesta: Elegir una imagen de cada pareja  1/2  2/2     1/2  2/2     1/2  2/2     1/2  2/2 [Pertinente]: 0
Pregunta: ¿Cuál de las imágenes emparejadas te resulta más atractiva?  Posibles valores de respuesta: Elegir una imagen de cada pareja  1/2  2/2     1/2  2/2     1/2  2/2     1/2  2/2 [Clara]: 0
Pregunta: ¿Cuál de las imágenes emparejadas te resulta más atractiva?  Posibles valores de respuesta: Elegir una imagen de cada pareja  1/2  2/2     1/2  2/2     1/2  2/2     1/2  2/2 [Relevante]: 0
Pregunta: ¿Qué te sugieren estas imágenes?  Posibles valores de respuesta: Respuesta abierta                   [Pertinente]: 0
Pregunta: ¿Qué te sugieren estas imágenes?  Posibles valores de respuesta: Respuesta abierta                   [Clara]: 0
Pregunta: ¿Qué te sugieren estas imágenes?  Posibles valores de respuesta: Respuesta abierta                   [Relevante]: 0
Pregunta: Cuando recibes/consumes imágenes... (selecciona la opción/opciones con la que más te identificas)  Posibles valores de respuesta (respuesta múltiple):  Observo si son bellas, armónicas o no 	Me planteo si significan algo para mí 	Observo el momento y contexto en el que aparecen   [Pertinente]: 0
Pregunta: Cuando recibes/consumes imágenes... (selecciona la opción/opciones con la que más te identificas)  Posibles valores de respuesta (respuesta múltiple):  Observo si son bellas, armónicas o no 	Me planteo si significan algo para mí 	Observo el momento y contexto en el que aparecen   [Clara]: 0
Pregunta: Cuando recibes/consumes imágenes... (selecciona la opción/opciones con la que más te identificas)  Posibles valores de respuesta (respuesta múltiple):  Observo si son bellas, armónicas o no 	Me planteo si significan algo para mí 	Observo el momento y contexto en el que aparecen   [Relevante]: 0
Pregunta: Señala las diferencias que existen entre estas imágenes con tus propias palabras  Posibles valores de respuesta: Respuesta abierta             [Pertinente]: 0
Pregunta: Señala las diferencias que existen entre estas imágenes con tus propias palabras  Posibles valores de respuesta: Respuesta abierta             [Clara]: 0
Pregunta: Señala las diferencias que existen entre estas imágenes con tus propias palabras  Posibles valores de respuesta: Respuesta abierta             [Relevante]: 0
Pregunta: ¿Te resultan atractivas estas imágenes? ¿Por qué?  Posibles valores de respuesta: Respuesta abierta         [Pertinente]: 0
Pregunta: ¿Te resultan atractivas estas imágenes? ¿Por qué?  Posibles valores de respuesta: Respuesta abierta         [Clara]: 0
Pregunta: ¿Te resultan atractivas estas imágenes? ¿Por qué?  Posibles valores de respuesta: Respuesta abierta         [Relevante]: 0
Pregunta: Cuando observas estas imágenes, ¿entiendes lo que pasa, aunque los sujetos no se muestren de forma completa?  Posibles valores de respuesta:  Sí 	No          [Pertinente]: 0
Pregunta: Cuando observas estas imágenes, ¿entiendes lo que pasa, aunque los sujetos no se muestren de forma completa?  Posibles valores de respuesta:  Sí 	No          [Clara]: 0
Pregunta: Cuando observas estas imágenes, ¿entiendes lo que pasa, aunque los sujetos no se muestren de forma completa?  Posibles valores de respuesta:  Sí 	No          [Relevante]: 0
Pregunta: Observa y describe lo que ves en las siguientes imágenes  Posibles valores de respuesta: Respuesta abierta           [Pertinente]: 0
Pregunta: Observa y describe lo que ves en las siguientes imágenes  Posibles valores de respuesta: Respuesta abierta           [Clara]: 0
Pregunta: Observa y describe lo que ves en las siguientes imágenes  Posibles valores de respuesta: Respuesta abierta           [Relevante]: 0
'''

columns_Perception = [131,132,133,
                      134,135,136,
                      137,138,139,
                      140,141,142,
                      143,144,145,
                      146,147,148,
                      149,150,151,
                      152,153,154,
                      155,156,157,
                      158,159,160,
                      161,162,163,
                      164,165,166,
                      167,168,169
                    ]

display(eResponsesNum.iloc[:,columns_Perception].describe())

display(eResponsesNum.iloc[:,columns_Perception].stack().describe())

,Pregunta: ¿Qué ves en estas imágenes? Posibles valores de respuesta: respuesta abierta [Pertinente],Pregunta: ¿Qué ves en estas imágenes? Posibles valores de respuesta: respuesta abierta [Clara],Pregunta: ¿Qué ves en estas imágenes? Posibles valores de respuesta: respuesta abierta [Relevante],Pregunta: ¿Qué forma/formas ves en estas imágenes? Posibles valores de respuesta: respuesta abierta [Pertinente],Pregunta: ¿Qué forma/formas ves en estas imágenes? Posibles valores de respuesta: respuesta abierta [Clara],Pregunta: ¿Qué forma/formas ves en estas imágenes? Posibles valores de respuesta: respuesta abierta [Relevante],Pregunta: ¿Has reconocido la/las formas rápidamente? Posibles valores de respuesta: Sí No [Pertinente],Pregunta: ¿Has reconocido la/las formas rápidamente? Posibles valores de respuesta: Sí No [Clara],Pregunta: ¿Has reconocido la/las formas rápidamente? Posibles valores de respuesta: Sí No [Relevante],"Pregunta: En las siguientes imágenes, ¿qué es lo que más te llama la atención? Posibles valores de respuesta: Respuesta abierta [Pertinente]","Pregunta: En las siguientes imágenes, ¿qué es lo que más te llama la atención? Posibles valores de respuesta: Respuesta abierta [Clara]","Pregunta: En las siguientes imágenes, ¿qué es lo que más te llama la atención? Posibles valores de respuesta: Respuesta abierta [Relevante]",Pregunta: ¿Qué ves en estas imágenes? Posibles valores de respuesta: Respuesta abierta [Pertinente],Pregunta: ¿Qué ves en estas imágenes? Posibles valores de respuesta: Respuesta abierta [Clara],Pregunta: ¿Qué ves en estas imágenes? Posibles valores de respuesta: Respuesta abierta [Relevante],Pregunta: ¿Qué ves en estas imágenes? Posibles valores de respuesta: Respuesta abierta [Pertinente],Pregunta: ¿Qué ves en estas imágenes? Posibles valores de respuesta: Respuesta abierta [Clara],Pregunta: ¿Qué ves en estas imágenes? Posibles valores de respuesta: Respuesta abierta [Relevante],Pregunta: ¿Cuál de las imágenes emparejadas te resulta más atractiva? Posibles valores de respuesta: Elegir una imagen de cada pareja 1/2 2/2 1/2 2/2 1/2 2/2 1/2 2/2 [Pertinente],Pregunta: ¿Cuál de las imágenes emparejadas te resulta más atractiva? Posibles valores de respuesta: Elegir una imagen de cada pareja 1/2 2/2 1/2 2/2 1/2 2/2 1/2 2/2 [Clara],Pregunta: ¿Cuál de las imágenes emparejadas te resulta más atractiva? Posibles valores de respuesta: Elegir una imagen de cada pareja 1/2 2/2 1/2 2/2 1/2 2/2 1/2 2/2 [Relevante],Pregunta: ¿Qué te sugieren estas imágenes? Posibles valores de respuesta: Respuesta abierta [Pertinente],Pregunta: ¿Qué te sugieren estas imágenes? Posibles valores de respuesta: Respuesta abierta [Clara],Pregunta: ¿Qué te sugieren estas imágenes? Posibles valores de respuesta: Respuesta abierta [Relevante],"Pregunta: Cuando recibes/consumes imágenes... (selecciona la opción/opciones con la que más te identificas) Posibles valores de respuesta (respuesta múltiple): Observo si son bellas, armónicas o no Me planteo si significan algo para mí Observo el momento y contexto en el que aparecen [Pertinente]","Pregunta: Cuando recibes/consumes imágenes... (selecciona la opción/opciones con la que más te identificas) Posibles valores de respuesta (respuesta múltiple): Observo si son bellas, armónicas o no Me planteo si significan algo para mí Observo el momento y contexto en el que aparecen [Clara]","Pregunta: Cuando recibes/consumes imágenes... (selecciona la opción/opciones con la que más te identificas) Posibles valores de respuesta (respuesta múltiple): Observo si son bellas, armónicas o no Me planteo si significan algo para mí Observo el momento y contexto en el que aparecen [Relevante]",Pregunta: Señala las diferencias que existen entre estas imágenes con tus propias palabras Posibles valores de respuesta: Respuesta abierta [Pertinente],Pregunta: Señala las diferencias que existen entre estas imágenes con tus propias palabras Posibles valores de respuesta: Respuesta abierta [Clara],Pregunta: Señala las di

count    194.000000
mean       0.989691
std        0.101271
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
dtype: float64

## Checking the evaluation of each theoretical construct


### Evaluation of the proposals for interpretation construct

In [21]:
'''
Pregunta: Cuando te encuentras con una imagen, ¿intentas entender con qué intención está hecha?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Pertinente]: 0
Pregunta: Cuando te encuentras con una imagen, ¿intentas entender con qué intención está hecha?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Clara]: 0
Pregunta: Cuando te encuentras con una imagen, ¿intentas entender con qué intención está hecha?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Relevante]: 0
Pregunta: Cuando recibes o consumes una imagen, ¿la comparas con tu propia realidad? ¿Intentas encontrar en tu entorno cierto parecido?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Pertinente]: 0
Pregunta: Cuando recibes o consumes una imagen, ¿la comparas con tu propia realidad? ¿Intentas encontrar en tu entorno cierto parecido?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Clara]: 0
Pregunta: Cuando recibes o consumes una imagen, ¿la comparas con tu propia realidad? ¿Intentas encontrar en tu entorno cierto parecido?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Relevante]: 0
Pregunta: Cuando recibes o consumes una imagen, ¿piensas en el modo en que fue obtenida (técnicamente)?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Pertinente]: 0
Pregunta: Cuando recibes o consumes una imagen, ¿piensas en el modo en que fue obtenida (técnicamente)?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Clara]: 0
Pregunta: Cuando recibes o consumes una imagen, ¿piensas en el modo en que fue obtenida (técnicamente)?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Relevante]: 0
Pregunta: Cuando observas una imagen, ¿te preguntas cosas acerca de los personajes que aparecen en la misma?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Pertinente]: 0
Pregunta: Cuando observas una imagen, ¿te preguntas cosas acerca de los personajes que aparecen en la misma?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Clara]: 0
Pregunta: Cuando observas una imagen, ¿te preguntas cosas acerca de los personajes que aparecen en la misma?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Relevante]: 0
Pregunta: ¿Entiendes generalmente las intenciones que tiene el autor de una imagen cuando la ves y la compartes?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Pertinente]: 0
Pregunta: ¿Entiendes generalmente las intenciones que tiene el autor de una imagen cuando la ves y la compartes?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Clara]: 0
Pregunta: ¿Entiendes generalmente las intenciones que tiene el autor de una imagen cuando la ves y la compartes?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Relevante]: 0
Pregunta: ¿Eres capaz de expresar verbalmente las emociones que te produce una imagen que has visto?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Pertinente]: 0
Pregunta: ¿Eres capaz de expresar verbalmente las emociones que te produce una imagen que has visto?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Clara]: 0
Pregunta: ¿Eres capaz de expresar verbalmente las emociones que te produce una imagen que has visto?  Posibles valores de respuesta:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre [Relevante]: 0
Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres?  Posibles valores de respuesta:  Sí 	No [Pertinente]: 0
Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres?  Posibles valores de respuesta:  Sí 	No [Clara]: 0
Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres?  Posibles valores de respuesta:  Sí 	No [Relevante]: 0
Pregunta: Si encuentras diferencias escribe cuales.  Posibles valores de respuesta: Respuesta abierta [Pertinente]: 0
Pregunta: Si encuentras diferencias escribe cuales.  Posibles valores de respuesta: Respuesta abierta [Clara]: 0
Pregunta: Si encuentras diferencias escribe cuales.  Posibles valores de respuesta: Respuesta abierta [Relevante]: 0
Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres?  Posibles valores de respuesta:   	Más mujeres que hombres 	 	 	Más hombres que mujeres. 	 	 	Los dos por igual  	  [Pertinente]: 0
Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres?  Posibles valores de respuesta:   	Más mujeres que hombres 	 	 	Más hombres que mujeres. 	 	 	Los dos por igual  	  [Clara]: 0
Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres?  Posibles valores de respuesta:   	Más mujeres que hombres 	 	 	Más hombres que mujeres. 	 	 	Los dos por igual  	  [Relevante]: 0
Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías?  Posibles valores de respuesta:   	Sí 	 	 	No [Pertinente]: 0
Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías?  Posibles valores de respuesta:   	Sí 	 	 	No [Clara]: 0
Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías?  Posibles valores de respuesta:   	Sí 	 	 	No [Relevante]: 0
Pregunta: Pon un ejemplo  Posibles valores de respuesta: Respuesta abierta [Pertinente]: 0
Pregunta: Pon un ejemplo  Posibles valores de respuesta: Respuesta abierta [Clara]: 0
Pregunta: Pon un ejemplo  Posibles valores de respuesta: Respuesta abierta [Relevante]: 0
'''

columns_Interpretation = [171,172,173,
                          174,175,176,
                          177,178,179,
                          180,181,182,
                          183,184,185,
                          186,187,188,
                          189,190,191,
                          192,193,194,
                          195,196,197,
                          198,199,200,
                          201,202,203
                         ]

display(eResponsesNum.iloc[:,columns_Interpretation].describe())

display(eResponsesNum.iloc[:,columns_Interpretation].stack().describe())

,"Pregunta: Cuando te encuentras con una imagen, ¿intentas entender con qué intención está hecha? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente]","Pregunta: Cuando te encuentras con una imagen, ¿intentas entender con qué intención está hecha? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara]","Pregunta: Cuando te encuentras con una imagen, ¿intentas entender con qué intención está hecha? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Relevante]","Pregunta: Cuando recibes o consumes una imagen, ¿la comparas con tu propia realidad? ¿Intentas encontrar en tu entorno cierto parecido? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente]","Pregunta: Cuando recibes o consumes una imagen, ¿la comparas con tu propia realidad? ¿Intentas encontrar en tu entorno cierto parecido? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara]","Pregunta: Cuando recibes o consumes una imagen, ¿la comparas con tu propia realidad? ¿Intentas encontrar en tu entorno cierto parecido? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Relevante]","Pregunta: Cuando recibes o consumes una imagen, ¿piensas en el modo en que fue obtenida (técnicamente)? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente]","Pregunta: Cuando recibes o consumes una imagen, ¿piensas en el modo en que fue obtenida (técnicamente)? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara]","Pregunta: Cuando recibes o consumes una imagen, ¿piensas en el modo en que fue obtenida (técnicamente)? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Relevante]","Pregunta: Cuando observas una imagen, ¿te preguntas cosas acerca de los personajes que aparecen en la misma? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente]","Pregunta: Cuando observas una imagen, ¿te preguntas cosas acerca de los personajes que aparecen en la misma? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara]","Pregunta: Cuando observas una imagen, ¿te preguntas cosas acerca de los personajes que aparecen en la misma? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Relevante]",Pregunta: ¿Entiendes generalmente las intenciones que tiene el autor de una imagen cuando la ves y la compartes? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente],Pregunta: ¿Entiendes generalmente las intenciones que tiene el autor de una imagen cuando la ves y la compartes? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara],Pregunta: ¿Entiendes generalmente las intenciones que tiene el autor de una imagen cuando la ves y la compartes? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Relevante],Pregunta: ¿Eres capaz de expresar verbalmente las emociones que te produce una imagen que has visto? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente],Pregunta: ¿Eres capaz de expresar verbalmente las emociones que te produce una imagen que has visto? Posibles valores de respuesta: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara],Pregunta: ¿Eres capaz de expresar verbalmente las emociones que te produce una imagen que has visto? Posibles valores de respues

count    157.000000
mean       0.980892
std        0.137344
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
dtype: float64

## Checking the evaluation of each cultural sub-construct


### Evaluation of the cultural items within the interpretation construct

In [22]:
'''
Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres?  Posibles valores de respuesta:  Sí 	No [Pertinente]: 0
Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres?  Posibles valores de respuesta:  Sí 	No [Clara]: 0
Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres?  Posibles valores de respuesta:  Sí 	No [Relevante]: 0

Pregunta: Si encuentras diferencias escribe cuales.  Posibles valores de respuesta: Respuesta abierta [Pertinente]: 0
Pregunta: Si encuentras diferencias escribe cuales.  Posibles valores de respuesta: Respuesta abierta [Clara]: 0
Pregunta: Si encuentras diferencias escribe cuales.  Posibles valores de respuesta: Respuesta abierta [Relevante]: 0

Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres?  Posibles valores de respuesta:   	Más mujeres que hombres 	 	 	Más hombres que mujeres. 	 	 	Los dos por igual  	  [Pertinente]: 0
Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres?  Posibles valores de respuesta:   	Más mujeres que hombres 	 	 	Más hombres que mujeres. 	 	 	Los dos por igual  	  [Clara]: 0
Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres?  Posibles valores de respuesta:   	Más mujeres que hombres 	 	 	Más hombres que mujeres. 	 	 	Los dos por igual  	  [Relevante]: 0

Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías?  Posibles valores de respuesta:   	Sí 	 	 	No [Pertinente]: 0
Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías?  Posibles valores de respuesta:   	Sí 	 	 	No [Clara]: 0
Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías?  Posibles valores de respuesta:   	Sí 	 	 	No [Relevante]: 0

Pregunta: Pon un ejemplo  Posibles valores de respuesta: Respuesta abierta [Pertinente]: 0
Pregunta: Pon un ejemplo  Posibles valores de respuesta: Respuesta abierta [Clara]: 0
Pregunta: Pon un ejemplo  Posibles valores de respuesta: Respuesta abierta [Relevante]: 0
'''

columns_Interpretation_MascFem = [189,190,191,
                                  192,193,194,
                                  195,196,197,
                                  198,199,200,
                                  201,202,203
                                 ]

display(eResponsesNum.iloc[:,columns_Interpretation_MascFem].describe())

display(eResponsesNum.iloc[:,columns_Interpretation_MascFem].stack().describe())

,Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres? Posibles valores de respuesta: Sí No [Pertinente],Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres? Posibles valores de respuesta: Sí No [Clara],Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres? Posibles valores de respuesta: Sí No [Relevante],Pregunta: Si encuentras diferencias escribe cuales. Posibles valores de respuesta: Respuesta abierta [Pertinente],Pregunta: Si encuentras diferencias escribe cuales. Posibles valores de respuesta: Respuesta abierta [Clara],Pregunta: Si encuentras diferencias escribe cuales. Posibles valores de respuesta: Respuesta abierta [Relevante],Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres? Posibles valores de respuesta: Más mujeres que hombres Más hombres que mujeres. Los dos por igual [Pertinente],Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres? Posibles valores de respuesta: Más mujeres que hombres Más hombres que mujeres. Los dos por igual [Clara],Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres? Posibles valores de respuesta: Más mujeres que hombres Más hombres que mujeres. Los dos por igual [Relevante],Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías? Posibles valores de respuesta: Sí No [Pertinente],Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías? Posibles valores de respuesta: Sí No [Clara],Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías? Posibles valores de respuesta: Sí No [Relevante],Pregunta: Pon un ejemplo Posibles valores de respuesta: Respuesta abierta [Pertinente],Pregunta: Pon un ejemplo Posibles valores de respuesta: Respuesta abierta [Clara],Pregunta: Pon un ejemplo Posibles valores de respuesta: Respuesta abierta [Relevante]
count,4.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,5.0,5.000000,5.000000,5.000000
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.800000,0.800000,0.800000
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.447214,0.447214,0.447214
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000


count    67.000000
mean      0.955224
std       0.208373
min       0.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
dtype: float64

## Checking the evaluation of cultural items


### Evaluation of the cultural items related to Masculinity vs. Feminity across all the questionnaire

In [23]:
'''
Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre  	  [Pertinente]: 0
Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre  	  [Clara]: 0
Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)?  Posibles respuestas:   	Nunca 	 	 	Muy pocas veces 	 	 	En ocasiones 	 	 	Bastantes veces 	 	 	Casi siempre o siempre  	  [Relevante]: 0


Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres?  Posibles valores de respuesta:  Sí 	No [Pertinente]: 0
Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres?  Posibles valores de respuesta:  Sí 	No [Clara]: 0
Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres?  Posibles valores de respuesta:  Sí 	No [Relevante]: 0

Pregunta: Si encuentras diferencias escribe cuales.  Posibles valores de respuesta: Respuesta abierta [Pertinente]: 0
Pregunta: Si encuentras diferencias escribe cuales.  Posibles valores de respuesta: Respuesta abierta [Clara]: 0
Pregunta: Si encuentras diferencias escribe cuales.  Posibles valores de respuesta: Respuesta abierta [Relevante]: 0

Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres?  Posibles valores de respuesta:   	Más mujeres que hombres 	 	 	Más hombres que mujeres. 	 	 	Los dos por igual  	  [Pertinente]: 0
Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres?  Posibles valores de respuesta:   	Más mujeres que hombres 	 	 	Más hombres que mujeres. 	 	 	Los dos por igual  	  [Clara]: 0
Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres?  Posibles valores de respuesta:   	Más mujeres que hombres 	 	 	Más hombres que mujeres. 	 	 	Los dos por igual  	  [Relevante]: 0

Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías?  Posibles valores de respuesta:   	Sí 	 	 	No [Pertinente]: 0
Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías?  Posibles valores de respuesta:   	Sí 	 	 	No [Clara]: 0
Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías?  Posibles valores de respuesta:   	Sí 	 	 	No [Relevante]: 0

Pregunta: Pon un ejemplo  Posibles valores de respuesta: Respuesta abierta [Pertinente]: 0
Pregunta: Pon un ejemplo  Posibles valores de respuesta: Respuesta abierta [Clara]: 0
Pregunta: Pon un ejemplo  Posibles valores de respuesta: Respuesta abierta [Relevante]: 0
'''

# Masculinity vs feminity

print("'Masculinity vs feminity' items")

columns_All_MascFem = [88,89,90,
                       189,190,191,
                       192,193,194,
                       195,196,197,
                       198,199,200,
                       201,202,203
                      ]

display(eResponsesNum.iloc[:,columns_All_MascFem].describe())

display(eResponsesNum.iloc[:,columns_All_MascFem].stack().describe())

'Masculinity vs feminity' items


,"Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Pertinente]","Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Clara]","Pregunta: Cuando haces fotos desde cualquiera de los medios a tu alcance, ¿te facilitan el proceso las opciones que te ofrecen las aplicaciones (aplicación de filtros, recortes, edición mediante grafismo...)? Posibles respuestas: Nunca Muy pocas veces En ocasiones Bastantes veces Casi siempre o siempre [Relevante]",Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres? Posibles valores de respuesta: Sí No [Pertinente],Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres? Posibles valores de respuesta: Sí No [Clara],Pregunta: ¿Crees que las fotografías hechas por hombres son diferentes a las que hacen las mujeres? Posibles valores de respuesta: Sí No [Relevante],Pregunta: Si encuentras diferencias escribe cuales. Posibles valores de respuesta: Respuesta abierta [Pertinente],Pregunta: Si encuentras diferencias escribe cuales. Posibles valores de respuesta: Respuesta abierta [Clara],Pregunta: Si encuentras diferencias escribe cuales. Posibles valores de respuesta: Respuesta abierta [Relevante],Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres? Posibles valores de respuesta: Más mujeres que hombres Más hombres que mujeres. Los dos por igual [Pertinente],Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres? Posibles valores de respuesta: Más mujeres que hombres Más hombres que mujeres. Los dos por igual [Clara],Pregunta: ¿Crees que en la creación de imágenes trabajan más mujeres u hombres? Posibles valores de respuesta: Más mujeres que hombres Más hombres que mujeres. Los dos por igual [Relevante],Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías? Posibles valores de respuesta: Sí No [Pertinente],Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías? Posibles valores de respuesta: Sí No [Clara],Pregunta: ¿Conoces autores/as que trabajen haciendo fotografías? Posibles valores de respuesta: Sí No [Relevante],Pregunta: Pon un ejemplo Posibles valores de respuesta: Respuesta abierta [Pertinente],Pregunta: Pon un ejemplo Posibles valores de respuesta: Respuesta abierta [Clara],Pregunta: Pon un ejemplo Posibles valores de respuesta: Respuesta abierta [Relevante]
count,5.0,4.0,4.0,4.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,5.0,5.000000,5.000000,5.000000
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.800000,0.800000,0.800000
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.447214,0.447214,0.447214
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000


count    80.000000
mean      0.962500
std       0.191182
min       0.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
dtype: float64

## Checking the evaluation of each theoretical construct


### Evaluation of the proposals for the general evaluation

In [24]:
'''
Adecuación. ¿El instrumento propuesto se adecúa a la evaluación/observación de la alfabetización visual?   : 0
Validez de la herramienta. ¿Las dimensiones propuestas, en conjunto, ayudan en la observación de la alfabetización visual?   : 0
Relevancia de las dimensiones. ¿Cada una de las dimensiones colabora en la explicación del proceso de alfabetización visual?   : 0
Puntos fuertes/débiles. Aspectos en los que se basa la solidez del constructo conformado; deficiencias a subsanar.: 0
'''

columns_Overview = [205,206,207]

display(eResponsesNum.iloc[:,columns_Overview].describe())

display(eResponsesNum.iloc[:,columns_Overview].stack().describe())

,Adecuación. ¿El instrumento propuesto se adecúa a la evaluación/observación de la alfabetización visual?,"Validez de la herramienta. ¿Las dimensiones propuestas, en conjunto, ayudan en la observación de la alfabetización visual?",Relevancia de las dimensiones. ¿Cada una de las dimensiones colabora en la explicación del proceso de alfabetización visual?
count,5.0,5.0,5.0
mean,1.0,1.0,1.0
std,0.0,0.0,0.0
min,1.0,1.0,1.0
25%,1.0,1.0,1.0
50%,1.0,1.0,1.0
75%,1.0,1.0,1.0
max,1.0,1.0,1.0


count    15.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
dtype: float64